# Exemplo de redes neurais: previsão de inadimplência

O **problema** que utilizaremos para exemplificar é o seguinte:

Deseja-se definir se um empréstimo será totalmente pago, classificando como *adimplente (bom/good)* ou *inadimplente (ruim/bad)*.
Os dados utilizados para abordar esse problema são os seguintes:

* Conjunto de dados Lending Club Loan Data disponibilizado [neste link](https://modeldata.tidymodels.org/reference/lending_club.html), contendo uma amostra de dados referente ao primeiro trimestre de 2016.
* Um dicionário sobre as variáveis pode ser conferido [neste link](https://resources.lendingclub.com/LCDataDictionary.xlsx).

Não houve nenhum pré-processamento nos dados.

Uma pequena análise exploratória é exibida abaixo:

# Biblioteca

In [1]:
# instalar bibliotecas

!pip install skimpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.1/116.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.1/819.1 kB 37.7 MB/s eta 0:00:00
  Attempting uninstall: typing-exte

In [1]:
# carregar bibliotecas

import pandas as pd
from skimpy import skim
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

# Dados

In [19]:
# Carregar os dados

data = pd.read_csv("dados-1.csv")
data

,funded_amnt,term,int_rate,sub_grade,addr_state,verification_status,annual_inc,emp_length,delinq_2yrs,inq_last_6mths,...,open_il_12m,open_il_24m,total_bal_il,all_util,inq_fi,inq_last_12m,delinq_amnt,num_il_tl,total_il_high_credit_limit,Class
0,16100,term_36,13.99,C4,CT,Not_Verified,35000.0,emp_5,0,0,...,0,0,1099,48,0,3,0,3,13193,good
1,32000,term_60,11.99,C1,MN,Verified,72000.0,emp_ge_10,0,0,...,1,3,49187,77,0,0,0,9,65945,good
2,10000,term_36,16.29,D1,OH,Source_Verified,72000.0,emp_ge_10,0,2,...,2,3,33378,79,1,4,0,9,39387,good
3,16800,term_60,13.67,C3,NV,Verified,101000.0,emp_lt_1,0,0,...,1,2,55445,64,1,4,0,10,60188,good
4,3500,term_36,7.39,A4,CA,Source_Verified,50100.0,emp_unk,0,0,...,0,0,0,78,0,0,0,3,0,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9852,3025,term_36,13.67,C3,GA,Not_Verified,43000.0,emp_ge_10,0,0,...,0,0,0,67,0,0,0,0,0,good
9853,16000,term_60,16.59,D2,OR,Not_Verified,32000.0,emp_3,0,1,...,0,1,12519,40,2,2,0,2,15051,good
9854,3000,term_36,9.16,B2,NY,Verified,18000.0,emp_lt_1,0,0,...,0,1,1109,73,0,0,0,2,1800,good
9855,35000,term_36,7.89,A5,OH,Source_Verified,240000.0,emp_ge_10,1,1,...,0,0,0,70,0,2,0,6,0,good


In [3]:
# uma pequena análise exploratória

skim(data)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 9857   │ │ int64       │ 14    │                                                          │
│ │ Number of columns │ 23     │ │ string      │ 6     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 3     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name            ┃ NA  ┃ NA %  ┃ mean    ┃ sd     ┃ p0    ┃ p25    ┃ p50   ┃ p75   ┃ p100   ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━┩  │
│ │ funded_amnt            │   0 │     0 │   16000 │   8900 │  1000 │   8500 │ 15000 │ 21000 │  40000 │ ▆▇▇▃▂▂ │  │
│ │ int_rate               │   0 │     0 │      13 │    4.9 │   5.3 │    8.5 │    12 │    15 │     29 │ ▇▇▅▃▁  │  │
│ │ annual_inc             │   0 │     0 │   80000 │  53000 │     0 │  50000 │ 69000 │ 96000 │ 960000 │   ▇    │  │
│ │ delinq_2yrs            │   0 │     0 │    0.33 │   0.89 │     0 │      0 │     0 │     0 │     22 │   ▇    │  │
│ │ inq_last_6mths         │   0 │     0 │    0.58 │   0.88 │     0 │      0 │     0 │     1 │      5 │  ▇▃▁   │  │
│ │ revol_util             │   0 │     0 │      52 │     24 │     0 │     33 │    52 │    70 │    140 │ ▃▇▇▅▁  │  │
│ │ acc_now_delinq         │   0 │     0 │  0.0063 │  0.083 │     0 │      0 │     0 │     0 │      2 │   ▇    │  │
│ │ open_il_6m             │   0 │     0 │     2.8 │    2.9 │     0 │      1 │     2 │     3 │     32 │   ▇▁   │  │
│ │ open_il_12m            │   0 │     0 │    0.74 │      1 │     0 │      0 │     0 │     1 │     20 │   ▇    │  │
│ │ open_il_24m            │   0 │     0 │     1.6 │    1.7 │     0 │      0 │     1 │     2 │     30 │   ▇    │  │
│ │ total_bal_il           │   0 │     0 │   35000 │  42000 │     0 │   9400 │ 24000 │ 46000 │ 590000 │   ▇    │  │
│ │ all_util               │   0 │     0 │      60 │     20 │     0 │     47 │    62 │    75 │    200 │  ▂▇▇   │  │
│ │ inq_fi                 │   0 │     0 │    0.93 │    1.5 │     0 │      0 │     0 │     1 │     15 │   ▇▁   │  │
│ │ inq_last_12m           │   0 │     0 │     2.2 │    2.4 │     0 │      0 │     2 │     3 │     32 │   ▇▁   │  │
│ │ delinq_amnt            │   0 │     0 │      12 │    570 │     0 │      0 │     0 │     0 │  42000 │   ▇    │  │
│ │ num_il_tl              │   0 │     0 │     8.6 │    7.5 │     0 │      4 │     7 │    11 │     82 │   ▇▂   │  │
│ │ total_il_high_credit_l │   0 │     0 │   45000 │  45000 │     0 │  16000 │ 34000 │ 61000 │ 550000 │   ▇▁   │  │
│ │ imit                   │     │       │         │        │       │        │       │       │        │        │  │
│ └────────────────────────┴─────┴───────┴─────────┴────────┴───────┴────────┴───────┴───────┴────────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                          ┃ NA    ┃ NA % 

As classificações produzidas pelo algoritmo são exibidas abaixo:

# Modelo

## Separação de amostras

In [8]:
#separação de amostras

training_data, test_data = train_test_split(
    data.filter(items = ["Class",
                         "funded_amnt",
                         "annual_inc",
                         "dlinq_2yrs",
                         "open_il_24m"],
                axis = "columns"),
    test_size = 0.3,
    random_state= 1984,
    stratify= data.Class
)

print(training_data)
print(test_data)

     Class  funded_amnt  annual_inc  open_il_24m
5715  good         8000     65000.0            1
2594  good         9975     46000.0            0
8890  good         5000     70800.0            1
3472   bad        14400     45000.0            1
4888  good         5000     23000.0            0
...    ...          ...         ...          ...
9646  good         7500     75000.0            2
4279  good         5000     44000.0            1
2463  good        10550     24000.0            1
5686  good        12300     82000.0            0
2815  good        12000     55000.0            1

[6899 rows x 4 columns]
     Class  funded_amnt  annual_inc  open_il_24m
1618  good        16000    143000.0            5
3492  good        35300    250000.0            5
6815  good        24000     84000.0            2
7765  good        17000     85987.0            2
8998  good        34425    168000.0            1
...    ...          ...         ...          ...
566   good        20900     63000.0         

In [15]:
# Treino do algoritmo
model = MLPClassifier()
model.fit(
    X = training_data.drop("Class", axis = "columns"),
    y = training_data.Class.values.ravel()
    )

MLPClassifier()

# Previsão

In [16]:
# produzir previsões

predict = model.predict(test_data.drop("Class", axis = "columns"))
predict

array(['good', 'good', 'good', ..., 'good', 'good', 'good'], dtype='<U4')

Por fim, reportamos algumas **medidas de acurácia**

In [21]:
#Calcular acurácia

print(confusion_matrix(test_data.Class,
                 predict))
print(accuracy_score(test_data.Class, predict))

[[   0  155]
 [   0 2803]]
0.9475997295469912


Como os dados são desbalanceados, isto é, há mais registros adimplentes do que inadimplentes, o modelo preveu que todos os registros serã adimplentes.

O que não é o adequado. Abaixo, o código provando que todas as linhas do objeto `predict` possuem uma unica categoria, que é a `good` (bom, adimplente)

In [25]:
pd.Series(predict).unique()

array(['good'], dtype=object)